<h1 style="color: blue">HD Wallet</h1>

## TOC:
* Mnemonic Seed
    * [Entropy](#entropy)
    * [Mnemonic creation](#mnemonic-create)
    * [Seeding](#seeding)
    * [Validation](#validation)
        * [Mnemonic validation](#mnemonic-validation)
        * [Seed validation](#seed-validation)

In [348]:
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes, hmac
from bitarray import bitarray, util
from secrets import token_bytes

## Entropy <a clas="anchor" id="entropy"></a>
Generate a blob between 128 and 256 bits from a secure random source.
The length of the entropy should be multiple of 32.

In [349]:
def entropy_gen(n = 128):
    if n < 128 or n > 256:
        raise Exception("Entropy size must be between 128 and 256 bits inclusive.")
    if n % 32 != 0:
        raise Exception("Entropy size must be multiple of 32.")
    return token_bytes(n // 8)

entropy = entropy_gen(32*8)
print(f"Entropy (hex): {entropy.hex()}") #256 bits

Entropy (hex): 10537cc9db047916503c94e8ca96560d716e4fc88196769c6c86a14c5fcc6265


## Mnemonic creation <a class="anchor" id="mnemonic-create"></a>

To make the entropy more human friendly we conver it to a series or words and introduce some error checking. To do so in this case we are gonna make use of __[BIP39-english](https://raw.githubusercontent.com/otromimi/bitcoin_testpad/wallet/BIP-0039_english.txt)__.

<div style="color:orange">
<h3>Atention</h3>
Mnemonic phrases are suported in many lenguages and alphabets. This notebook only uses the english ASCII variant. For other alphabets use UTF-8 NFKD.
</div>

In [389]:
colors = {'purple':'\033[95m', 'red':'\033[91m', 'yellow':'\033[93m', 'green':'\033[92m', 'blue':'\033[94m'}

def mnemonic(bits):
    # Entorpy 2 binary
    entropy_bits = bitarray()
    entropy_bits.frombytes(bits)
    
    # Checksum
    hash256 = hashes.Hash(hashes.SHA256())
    hash256.update(bits)
    bin_h = bitarray()
    bin_h.frombytes(hash256.finalize())
    
    entropy_bits = entropy_bits + bin_h[:(len(entropy_bits) // 32)]

    numbers = [] # words numbers
    for i in range(len(entropy_bits)//11):
        numbers.append(util.ba2int(entropy_bits[11*i:11*i+11]))

    words = {} # words in our entropy
    with open('BIP-0039_english.txt', 'r') as file:
        for i, word in enumerate(file):
            if i in numbers:
                words[i] = word
                if len(numbers) == len(words):
                    break
    
    mnemonic_sentence = [] # mneumonic
    for i in numbers:
        mnemonic_sentence.append(words[i].strip()) 
    
    return tuple(mnemonic_sentence)


# change this assigment to check for different values.
entropy = entropy # Ex: entropy = entropy_gen()

mnemonic_sentence = " ".join(mnemonic(entropy))

# Entropy binary
entropy_bits = bitarray()
entropy_bits.frombytes(entropy)

# Entropy hash
hash256 = hashes.Hash(hashes.SHA256())
hash256.update(entropy)
bin_h = bitarray()
entropy_hash = hash256.finalize()
bin_h.frombytes(entropy_hash)

print(colors['purple']+entropy_bits.to01(), end="")
print(colors['yellow']+bin_h.to01()[:(len(entropy_bits) // 32)], end="\n")
print(f"{colors['purple']}\u2589 {'Entropy'}")
print(f"{colors['yellow']}\u2589 {'Checksum (hash)'}\033[0m\n")

print(f"Entropy (hex): {entropy.hex()}")
print(f"entropy SHA-256: {entropy_hash.hex()}")

print("\n\u250F"+"\u2501"*(len(mnemonic_sentence)+2)+"\u2513")
print("\u2503 "+mnemonic_sentence+" \u2503")
print("\u2517"+"\u2501"*(len(mnemonic_sentence)+2)+"\u251B")

del entropy_bits, hash256, bin_h, entropy_hash


000100000101001101111100110010011101101100000100011110010001011001010000001111001001010011101000110010101001011001010110000011010111000101101110010011111100100010000001100101100111011010011100011011001000011010100001010011000101111111001100011000100110010110010000
▉ Entropy
▉ Checksum (hash)

Entropy (hex): 10537cc9db047916503c94e8ca96560d716e4fc88196769c6c86a14c5fcc6265
entropy SHA-256: 90e664efd323e9eb623f0eaa6f65c3adb2eaf7a2b0f98ee1ebbf027278a6dbde

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ aware orbit crash render elegant menu domain myth trip feed night brain black child capital crazy isolate toddler canvas dream shine tower maze rare ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛


## Seeding<a class="anchor" id="seeding"></a>
Derivates the seed that will be use for creating our cryptographic keys. We achive this by using an algorithm computer intensive that will slow down an attack.

Algorithm used: PBKDF2_HMAC (Password Based Key Derivation Function 2)</br>
* Hashing function: SHA-512 HMAC</br>
* Iterations: 2048</br>
* length: resulting key (seed) lenght in bytes</br>

Thanks to the use of a HMAC (Hash Message Authentication Code) hash function we can add a salt (password) to our nmonic phrase.



In [409]:
def mnemonic2seed(mnemonic_sentence, salt=""):
    
    mnemonic_dev = PBKDF2HMAC(
        algorithm = hashes.SHA512(),
        length = 64, #bytes
        salt = ("mnemonic"+salt).encode('ascii'),
        iterations = 2048
    )

    mnemonic_bytes = mnemonic_sentence.encode('ascii')
    seed = mnemonic_dev.derive(mnemonic_bytes)

    return seed


recovery_pass = "" # In case we want to add a salt to our mneumonic, add in here

seed = mnemonic2seed(mnemonic_sentence, recovery_pass)

if recovery_pass:
    print(f"Recovery password: {colors['red']}{recovery_pass}\033[0m")
print(f"Seed: {seed.hex()}")

Seed: 4ace8ee3dc0722636b2ef3911ab2f69667979e36fc9aeb3653a065b7c22fe94f90093c8eb8afedaa268bf6dd8dd3bda19ef4b6f7c81bd8e811968f4bfdfb3737


## Validation<a class="anchor" id="validation"></a>

### Mnemonic validation<a class="anchor" id="mnemonic-validation"></a>
Validates de entropy against its hash to determinate if the mnemonic secuence was correct.

Requires: __[BIP39-english](https://raw.githubusercontent.com/otromimi/bitcoin_testpad/wallet/BIP-0039_english.txt)__.

In [388]:
def mnemonic_check(mnemonic):

    words = mnemonic.split(" ")
    numbers = [None] * len(words)

    with open('BIP-0039_english.txt') as file:
        for i, word in enumerate(file):
            if word.strip() in words:
                for j, item in enumerate(words):
                    if item == word.strip():
                        numbers[j] = i
    
    entropy = "".join([f'{i:011b}' for i in numbers])
    entropy = bitarray(entropy)

    checksum_len = len(entropy) % 32

    hash256 = hashes.Hash(hashes.SHA256())
    hash256.update(entropy[:-checksum_len].tobytes())
    bin_h = bitarray()
    entropy_hash = hash256.finalize()
    bin_h.frombytes(entropy_hash)
    
    if entropy[-checksum_len:] != bin_h[:checksum_len]:
        # Just raised in case checksum for mneumonic fails.
        raise Exception(f"Checksum fail, hash: {entropy_hash.hex()}")
    
    return entropy[:-checksum_len].tobytes()



recover_entropy = mnemonic_check(mnemonic_sentence)

print(f"{entropy.hex()} -> entropy") # entropy generated on first cell
print(f"{recover_entropy.hex()} -> recovered entropy")


10537cc9db047916503c94e8ca96560d716e4fc88196769c6c86a14c5fcc6265 -> entropy
10537cc9db047916503c94e8ca96560d716e4fc88196769c6c86a14c5fcc6265 -> recovered entropy


### Seed validation<a class="anchor" id="seed-validation"></a>

Validates the seed against the entropy and password.

In [417]:
def seed_check(seed, password):

    seed_chk = PBKDF2HMAC(
        algorithm = hashes.SHA512(),
        length = 64,
        salt = ("mnemonic"+password).encode('ascii'),
        iterations = 2048
    )
    seed_chk.verify(mnemonic_sentence.encode('ascii'), seed) # in case validation fails will raise a exception

if not seed_check(seed, recovery_pass):
    print("Seed check passed. \u2705")

Seed check passed. ✅
